# Model order reduction with artificial neural networks in pyMOR

## Overview
- Application: Parametrized PDEs (stationary and instationary)
- Theory: Reduced Basis Methods, Proper Orthogonal Decomposition, Artifical Neural Networks
- Practice: Implementation in pyMOR

In [1]:
import sys
if 'pyodide' in sys.modules:
    import piplite
    await piplite.install(['pymor', 'matplotlib', 'ipython'])

## A non-intrusive reduced order method using artificial neural networks

### Two scenarios:

- Given a full-order model $\mu\mapsto u(\mu)$, but e.g. no affine decomposition of operators
- Given only a set $\{(\mu_i,u(\mu_i))\}_{i=1}^n$ of parameter values with corresponding snapshots

### The approach:

- Compute a reduced space $V_N$ with (orthonormal) basis $\Psi_N$ using only snapshot data (via proper orthogonal decomposition)
- Project full-order solution $u(\mu)$ onto $V_N$ (orthogonal projection!):
\begin{align}
    \pi_N\colon\mathcal{P}\to\mathbb{R}^N,\qquad\pi_N(\mu)=\Psi_N^\top u(\mu)
\end{align}
- Function $\pi_N$ returns for a parameter $\mu\in\mathcal{P}$ the coefficients of the projection $u_N(\mu)$ of $u(\mu)$ onto $V_N$ w.r.t. the basis $\Psi_N$
- Approximate the map $\pi_N$ by a neural network $\Phi_N$

### Error estimate:

\begin{align}
    \lVert u(\mu)-\Psi_N\Phi_N(\mu)\rVert\leq\underbrace{\lVert u(\mu)-u_N(\mu)\rVert}_{\text{best-approximation error in }V_N} + \underbrace{\lVert\pi_N(\mu)-\Phi_N(\mu)\rVert}_{\text{approximation error of the neural network}}
\end{align}

### Available variants in pyMOR:

| Setting | Inputs | Outputs | Visualization |
| :-: | :-: | :-: | :-: |
| Stationary, State | $\mu$ | $\pi_N(\mu)$ | <img src="files/mu_to_coeffs.svg" alt="Feedforward neural network for model order reduction" width="100%" /> |
| Instationary, State | $(\mu,t)$ | $\pi_N$$(\mu,t)$ | <img src="files/mu_and_time_to_coeffs.svg" alt="Feedforward neural network for model order reduction" width="100%" /> |
| Stationary, Output | $\mu$ | $\mathcal{J}(\mu)$ | <img src="files/mu_to_output.svg" alt="Feedforward neural network for model order reduction" width="100%" /> |
| Instationary, Output | $(\mu,t)$ | $\mathcal{J}(\mu,t)$ | <img src="files/mu_and_time_to_output.svg" alt="Feedforward neural network for model order reduction" width="100%" /> |

## A stationary example in pyMOR

### Setting up the problem:

Example problem:
\begin{align}
    -\nabla \cdot \big(\sigma(x, \mu) \nabla u(x, \mu) \big) = f(x, \mu),\quad x\in \Omega=(0,1)^2,
\end{align}
with data functions 
\begin{align}
    f((x_1, x_2), \mu) &= 10 \cdot \mu + 0.1,\\
    \sigma((x_1, x_2), \mu) &= (1 - x_1) \cdot \mu + x_1,
\end{align}
where $\mu \in (0.1, 1)$ denotes the parameter. Further, we apply the Dirichlet boundary conditions

\begin{align}
    u((x_1, x_2), \mu) = 2x_1\mu + 0.5,\quad x=(x_1, x_2) \in \partial\Omega.
\end{align}

In [2]:
from pymor.core.logger import set_log_levels
set_log_levels({'pymor.discretizers.builtin.cg.DiffusionOperatorP1': 'ERROR', 'pymor.discretizers.builtin.cg.L2ProductP1': 'ERROR'})

In [3]:
from pymor.basic import *

problem = StationaryProblem(
      domain=RectDomain(),

      rhs=LincombFunction(
          [ExpressionFunction('10', 2), ConstantFunction(1., 2)],
          [ProjectionParameterFunctional('mu'), 0.1]),

      diffusion=LincombFunction(
          [ExpressionFunction('1 - x[0]', 2), ExpressionFunction('x[0]', 2)],
          [ProjectionParameterFunctional('mu'), 1]),

      dirichlet_data=LincombFunction(
          [ExpressionFunction('2 * x[0]', 2), ConstantFunction(1., 2)],
          [ProjectionParameterFunctional('mu'), 0.5]),

      name='2DProblem'
  )

fom, _ = discretize_stationary_cg(problem, diameter=1/50)

parameter_space = fom.parameters.space((0.1, 1))

### Setting up the neural network reductor:

In [4]:
training_set = parameter_space.sample_uniformly(100)
validation_set = parameter_space.sample_randomly(20)

In [5]:
from pymor.reductors.neural_network import NeuralNetworkReductor

nn_reductor = NeuralNetworkReductor(fom,
                                    training_set,
                                    validation_set,
                                    l2_err=1e-5,  # POD error
                                    ann_mse=1e-5)  # Neural network training error

*Alternative, purely data-driven usage:* Pass pairs of parameters and solutions as training/validation set to the reductor and set `fom=None`.

In [6]:
print(nn_reductor.reduce.__doc__)

Reduce by training artificial neural networks.

        Parameters
        ----------
        hidden_layers
            Number of neurons in the hidden layers. Can either be fixed or
            a Python expression string depending on the reduced basis size
            respectively output dimension `N` and the total dimension of
            the |Parameters| `P`.
        activation_function
            Activation function to use between the hidden layers.
        optimizer
            Algorithm to use as optimizer during training.
        epochs
            Maximum number of epochs for training.
        batch_size
            Batch size to use if optimizer allows mini-batching.
        learning_rate
            Step size to use in each optimization step.
        loss_function
            Loss function to use for training. If `'weighted MSE'`, a weighted
            mean squared error is used as loss function, where the weights are
            given as the singular values of the correspo

In [7]:
nn_rom = nn_reductor.reduce()

00:01 NeuralNetworkReductor: Computing training snapshots ...
00:01 |   StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.1]} ...
00:01 |   StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.1090909090909091]} ...
00:01 |   StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.1181818181818182]} ...
00:01 |   StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.1272727272727273]} ...
00:01 |   StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.13636363636363635]} ...
00:01 |   StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.14545454545454545]} ...
00:01 |   StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.15454545454545454]} ...
00:01 |   StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.16363636363636364]} ...
00:01 |   StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.17272727272727273]} ...
00:01 |   StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.18181818181818182]} ...
00:01 |   StationaryModel: 

In [8]:
print(f'Reduced basis size: {len(nn_reductor.reduced_basis)}')
print(f'Neural network losses: {nn_reductor.losses}')

Reduced basis size: 9
Neural network losses: {'full': 7.75532936792775e-06, 'train': 8.237686756278563e-06, 'val': 5.3435424261736885e-06}


### Test of the ROM:

In [9]:
mu = parameter_space.sample_randomly()

U = fom.solve(mu)
U_red = nn_rom.solve(mu)
U_red_recon = nn_reductor.reconstruct(U_red)

fom.visualize((U, U_red_recon, U-U_red_recon),
              legend=(f'Full solution for parameter {mu}', f'Reduced solution for parameter {mu}', f'Difference between solution and approximation'),
              separate_colorbars=True)

00:09 StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.7822789660768364]} ...
00:09 NeuralNetworkModel: Solving 2DProblem_CG_reduced for {input: , mu: [0.7822789660768364]} ...


Output()

### Error and runtime comparison:

In [10]:
test_set = parameter_space.sample_randomly(10)

In [11]:
import time
import numpy as np

def compute_average_errors_and_speedups(rom, reductor):
    U = fom.solution_space.empty(reserve=len(test_set))
    U_red = fom.solution_space.empty(reserve=len(test_set))

    speedups = []
    
    for mu in test_set:
        tic = time.perf_counter()
        U.append(fom.solve(mu))
        time_fom = time.perf_counter() - tic

        tic = time.perf_counter()
        U_red.append(reductor.reconstruct(rom.solve(mu)))
        time_red = time.perf_counter() - tic

        speedups.append(time_fom / time_red)
        
    absolute_errors = (U - U_red).norm()
    relative_errors = (U - U_red).norm() / U.norm()
    
    return np.average(absolute_errors), np.average(relative_errors), np.average(speedups)

In [12]:
avg_abs_err, avg_rel_err, avg_speedup = compute_average_errors_and_speedups(nn_rom, nn_reductor)

print(f'Average absolute error: {avg_abs_err}')
print(f'Average relative error: {avg_rel_err}')
print(f'Average speedup: {avg_speedup}')

00:10 StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.4190733713168815]} ...
00:10 NeuralNetworkModel: Solving 2DProblem_CG_reduced for {input: , mu: [0.4190733713168815]} ...
00:10 StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.973628221955413]} ...
00:10 NeuralNetworkModel: Solving 2DProblem_CG_reduced for {input: , mu: [0.973628221955413]} ...
00:10 StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.9038090091899779]} ...
00:10 NeuralNetworkModel: Solving 2DProblem_CG_reduced for {input: , mu: [0.9038090091899779]} ...
00:10 StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.8005451473663857]} ...
00:10 NeuralNetworkModel: Solving 2DProblem_CG_reduced for {input: , mu: [0.8005451473663857]} ...
00:10 StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.2751748370667708]} ...
00:10 NeuralNetworkModel: Solving 2DProblem_CG_reduced for {input: , mu: [0.2751748370667708]} ...
00:10 StationaryModel: Solving 2DProblem_CG for {input: , mu: [0.52

Average absolute error: 0.0068720104508320735
Average relative error: 8.960207935201087e-05
Average speedup: 8.249468919281586


In [13]:
set_log_levels({'pymor.models.basic.StationaryModel': 'ERROR'})

### Exercise (Revision of the `StationaryRBReductor` from the morning session):

Set up a `StationaryRBReductor` for the FOM using the same reduced basis as built by the `NeuralNetworkReductor` (the reduced basis can be accessed via `nn_reductor.reduced_basis`) and compute a corresponding ROM. Compute the errors and speedups for the test set from above.

### Exercise:

Create two lists of tuples of parameters and corresponding solutions of the FOM - one for the training set and one for the validation set. Create a new `NeuralNetworkReductor` with these lists as inputs instead of a FOM (use `l2_err=1e-5` and `ann_mse=1e-5` as above). Call the `reduce`-method of the reductor (it might be necessary to increase the number of restarts to train a neural network that reaches the prescribed tolerance) and evaluate the performance of the ROM compared to the FOM.

## Extending the problem by output quantities

In [14]:
problem = problem.with_(outputs=[('l2', problem.rhs), ('l2_boundary', problem.dirichlet_data)])

In [15]:
fom, _ = discretize_stationary_cg(problem, diameter=1/50)

In [16]:
from pymor.reductors.neural_network import NeuralNetworkStatefreeOutputReductor

output_reductor = NeuralNetworkStatefreeOutputReductor(fom,
                                                       training_set,
                                                       validation_set,
                                                       validation_loss=1e-5)

output_rom = output_reductor.reduce(log_loss_frequency=2)

00:10 NeuralNetworkStatefreeOutputReductor: Computing training samples ...
00:11 NeuralNetworkStatefreeOutputReductor: Computing validation snapshots ...
00:11 NeuralNetworkStatefreeOutputReductor: Training of neural network ...
00:11 |   NeuralNetworkStatefreeOutputReductor: Initializing neural network ...
00:11 |   FullyConnectedNN: Architecture of the neural network:
FullyConnectedNN(
  (layers): ModuleList(
    (0): Linear(in_features=1, out_features=9, bias=True)
    (1): Linear(in_features=9, out_features=9, bias=True)
    (2): Linear(in_features=9, out_features=2, bias=True)
  )
)
00:11 |   multiple_restarts_training: Performing up to 10 restarts to train a neural network with a loss below 1.000e-05 ...
00:11 |   multiple_restarts_training: Training neural network #0 ...
00:11 |   |   train_neural_network: Starting optimization procedure ...
00:11 |   |   train_neural_network: Epoch 0: Current train loss of 1.299e-02
00:11 |   |   train_neural_network: Epoch 0: Current val loss 

In [17]:
outputs = []
outputs_red = []
outputs_speedups = []

for mu in test_set:
    tic = time.perf_counter()
    outputs.append(fom.output(mu=mu))
    time_fom = time.perf_counter() - tic

    tic = time.perf_counter()
    outputs_red.append(output_rom.output(mu=mu))
    time_red = time.perf_counter() - tic
    
    outputs_speedups.append(time_fom / time_red)

outputs = np.squeeze(np.array(outputs))
outputs_red = np.squeeze(np.array(outputs_red))

outputs_absolute_errors = np.abs(outputs - outputs_red)
outputs_relative_errors = np.abs(outputs - outputs_red) / np.abs(outputs)

00:11 NeuralNetworkStatefreeOutputModel: Solving 2DProblem_CG_output_reduced for {input: , mu: [0.4190733713168815]} ...
00:11 NeuralNetworkStatefreeOutputModel: Solving 2DProblem_CG_output_reduced for {input: , mu: [0.973628221955413]} ...
00:11 NeuralNetworkStatefreeOutputModel: Solving 2DProblem_CG_output_reduced for {input: , mu: [0.9038090091899779]} ...
00:11 NeuralNetworkStatefreeOutputModel: Solving 2DProblem_CG_output_reduced for {input: , mu: [0.8005451473663857]} ...
00:11 NeuralNetworkStatefreeOutputModel: Solving 2DProblem_CG_output_reduced for {input: , mu: [0.2751748370667708]} ...
00:11 NeuralNetworkStatefreeOutputModel: Solving 2DProblem_CG_output_reduced for {input: , mu: [0.5200489033543307]} ...
00:12 NeuralNetworkStatefreeOutputModel: Solving 2DProblem_CG_output_reduced for {input: , mu: [0.13942338920850592]} ...
00:12 NeuralNetworkStatefreeOutputModel: Solving 2DProblem_CG_output_reduced for {input: , mu: [0.23886054286079306]} ...
00:12 NeuralNetworkStatefreeOut

In [18]:
print(f'Average absolute error: {np.average(outputs_absolute_errors)}')
print(f'Average relative error: {np.average(outputs_relative_errors)}')
print(f'Average speedup: {np.average(outputs_speedups)}')

Average absolute error: 0.0013971907838333686
Average relative error: 0.0004165336350060543
Average speedup: 11.294453029588952


### Exercise:

Set up a new `NeuralNetworkReductor` for the changed FOM (which now includes the output quantities) with the same tolerances. Use this new reductor to compute a ROM that also takes the output into account and compare the results on the test set with those from the `NeuralNetworkStatefreeOutputReductor`.

## Summary

- pyMOR provides highly customizable training routines for neural networks with various options and parameters to tune
- Implementation respects abstract interfaces, i.e. the reductors can be directly applied to models/solutions originating from external solvers (see for instance https://github.com/pymor/pymor/blob/main/src/pymordemos/neural_networks_instationary.py for an example of Navier-Stokes equations using the FEniCS bindings in pyMOR)
- Applicable to stationary and instationary problems
- Different architecture called long short-term memory (LSTM) neural networks is also available for instationary problems (this architecture is closely related to time-stepping schemes)